In [1]:
%%capture
!pip install unsloth
!pip install weave
!pip install python-dotenv
!pip install wandb
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [2]:
import unsloth
import weave
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

  # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
  "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
  "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
  "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
  "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
  "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
  "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
  "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
  "unsloth/Phi-3-medium-4k-instruct",
  "unsloth/gemma-2-9b-bnb-4bit",
  "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

  "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
  "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
  "unsloth/Llama-3.2-3B-bnb-4bit",
  "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

  "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

#### Configure Parameters
Create the model and tokenizer from a pretrained model and **if** we have a checkpoint path supplied as a parameter then load the adapter

In [3]:
def configure_parameters(resume_path=None):
  model_name = "unsloth/Llama-3.2-1B-Instruct"

  if resume_path: ###########REMOVE NOT
      # If you are resuming, you CANNOT change the base model.
      # See the important warning below.
      model_name = resume_path


  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = model_name,
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
  )

  return model, tokenizer


## Mount google drive to get checkpoint

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import json

drive_path = "/content/drive/My Drive/KTH Masters/ID2222 - Scalable/ckpt"

#comment below out for 3b
drive_path = "/content/drive/My Drive/KTH Masters/ID2222 - Scalable/1b/ckpt"

resume_path = None
model = None
tokenizer = None

Mounted at /content/drive


#### Checkpoint Loading Logic
Find the checkpoint folder and sort them based on the checkpoint schema "checkpoint-*".

We try to get the target_epoch which is represented by the latest checkpoint file. Then we make sure to use this file path as the `resume_path` needed for configuring the model

If we don't have a checkpoint then just create the base model with base configs.

After mounting your Google Drive, you can copy your checkpoint folder from Drive to the local `./ckpt` directory. **Please replace `/path/to/your/checkpoint_folder_on_drive` with the actual path to your checkpoint folder on Google Drive.**

In [25]:
if not os.path.exists(drive_path):
    print("Drive path does not exist. Fix the mount path.")
else:
    checkpoint_folders = sorted(
        [f for f in os.listdir(drive_path) if f.startswith("checkpoint-")],
        key=lambda x: int(x.split("-")[1])
    )

    if checkpoint_folders:
        resume_path = os.path.join(drive_path, checkpoint_folders[-1])
        print(f"Found checkpoint: {resume_path}")
    else:
        print("No checkpoints found — starting new run.")

# LOAD MODEL WITH OR WITHOUT ADAPTER
try:
    model, tokenizer = configure_parameters(resume_path=resume_path)
    if resume_path:
        print("✔ Adapter loaded — resuming training.")
    else:
        print("✔ Starting new training run.")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# APPLY LORA ONLY IF STARTING FRESH
if resume_path is None: ###########REMOVE NOT
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=3407,
        use_rslora=False,
        loftq_config=None,
    )
    print("✔ LoRA initialized (new run).")
else:
    print("✔ LoRA adapter restored from checkpoint.")

Found checkpoint: /content/drive/My Drive/KTH Masters/ID2222 - Scalable/1b/ckpt/checkpoint-563
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✔ Adapter loaded — resuming training.
✔ LoRA adapter restored from checkpoint.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset

dataset = load_dataset("mlabonne/FineTome-100k", split="train").shuffle(seed=42).select(range(5000))
# dataset = load_dataset("mlabonne/FineTome-3k", split="train")
# dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [7]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [8]:
dataset[5]["conversations"]

[{'content': 'How can I modify a C++ program to print out a Fibonacci sequence up to the nth number?',
  'role': 'user'},
 {'content': "You can modify the given C++ program to print out a Fibonacci sequence up to the nth number. First, you need to declare and initialize a variable 'n' to specify the number of terms you want in the sequence. In the given code, 'n' is assigned a value of 10. \n\nNext, you declare three variables: 'first', 'second', and 'next'. 'first' and 'second' are initialized to 0 and 1 respectively, as these are the first two terms of the Fibonacci sequence.\n\nThen, you can use a for loop to iterate 'n' times and calculate the Fibonacci sequence. Inside the loop, you check if the current index 'i' is less than or equal to 1. If it is, 'next' is assigned the value of 'i'. This is because the first two terms of the Fibonacci sequence are 0 and 1. \n\nIf 'i' is greater than 1, 'next' is calculated by adding 'first' and 'second'. 'first' is then updated to the previous

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [9]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow can I modify a C++ program to print out a Fibonacci sequence up to the nth number?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nYou can modify the given C++ program to print out a Fibonacci sequence up to the nth number. First, you need to declare and initialize a variable 'n' to specify the number of terms you want in the sequence. In the given code, 'n' is assigned a value of 10. \n\nNext, you declare three variables: 'first', 'second', and 'next'. 'first' and 'second' are initialized to 0 and 1 respectively, as these are the first two terms of the Fibonacci sequence.\n\nThen, you can use a for loop to iterate 'n' times and calculate the Fibonacci sequence. Inside the loop, you check if the current index 'i' is less than or equal to 1. If it is, 'next' is assigned the value of 'i'. Thi

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [10]:
# split dataset for evaluation per epoch
split_dataset = dataset.train_test_split(test_size = 0.1, seed = 3407)

train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

train_dataset = train_dataset.remove_columns(["conversations"])
eval_dataset  = eval_dataset.remove_columns(["conversations"])

In [11]:
from dotenv import load_dotenv  # Import the function
import os
import wandb

load_dotenv()

wandb_token = os.getenv("WANDB")
if wandb_token:
    wandb.login(key=wandb_token)
else:
    print("Warning: WANDB_API_KEY not found in .env file")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: georgz (georgz-kth-royal-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [12]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datetime import datetime

# Generate unique run name with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"sft-training-{timestamp}"

training_args = TrainingArguments(
    # === SPEED OPTIMIZATIONS ===
    per_device_train_batch_size=4,      # INCREASED: 2→4 (use more GPU memory)
    per_device_eval_batch_size=8,       # INCREASED: Can be even higher for eval
    gradient_accumulation_steps=2,      # DECREASED: 4→2 (still effective batch=8)
    gradient_checkpointing=True,        # Keep this, necessary for VRAM

    # === Learning Rate & Schedule ===
    learning_rate=3e-4,                 # INCREASED: Faster convergence
    lr_scheduler_type="cosine",
    warmup_ratio=0.01,                  # DECREASED: Less warmup = faster start

    # === Training Duration - REDUCE THIS! ===
    num_train_epochs=1,                 # REDUCED: 2→1 epoch (try this first!)
    max_steps=-1,                       # Or set max_steps=500 to limit total steps

    # === Precision ===
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),

    # === Optimizer ===
    optim="adamw_8bit",
    weight_decay=0.01,
    max_grad_norm=1.0,

    # === Logging & Evaluation ===
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=400,
    logging_strategy="steps",

    # === Saving ===
    save_strategy="steps",          # <-- save based on steps
    save_steps=500,                 # <-- checkpoint every N steps
    save_total_limit=2,             # keep a couple of recent checkpoints
    load_best_model_at_end=False,

    # === Output & Reporting ===
    output_dir=drive_path,
    report_to="wandb",
    run_name=run_name,

    # === Performance ===
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    group_by_length=True,               # CRITICAL for speed!
    dataloader_prefetch_factor=2,       # ADDED: Prefetch batches

    # === Misc ===
    seed=3407,
    remove_unused_columns=True  ,
    ddp_find_unused_parameters=False,
    disable_tqdm=False,                 # Keep progress bar to monitor speed
)

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from dotenv import load_dotenv
import wandb

data_collector = DataCollatorForSeq2Seq(
    tokenizer = tokenizer,
    padding=True
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = data_collector,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/4500 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=6):   0%|          | 0/4500 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

We verify masking is actually done:

In [15]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"How can magnetic resonance spectroscopy be used to determine the purity and concentration of a sample in organic chemistry?"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMagnetic Resonance Spectroscopy (MRS), specifically Nuclear Magnetic Resonance (NMR) spectroscopy, is a powerful analytical technique used in organic chemistry to determine the purity and concentration of a sample. It provides detailed information about the molecular structure, dynamics, and chemical environment of molecules in a sample. Here\'s how NMR spectroscopy can be used to determine the purity and concentration of a sample in organic chemistry:\n\n1. Principle of NMR spectroscopy: NMR spectroscopy is based on the principle that certain atomic nuclei, such as 1H (protons) and 13C (carbon-13), have a 

In [16]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                         Magnetic Resonance Spectroscopy (MRS), specifically Nuclear Magnetic Resonance (NMR) spectroscopy, is a powerful analytical technique used in organic chemistry to determine the purity and concentration of a sample. It provides detailed information about the molecular structure, dynamics, and chemical environment of molecules in a sample. Here's how NMR spectroscopy can be used to determine the purity and concentration of a sample in organic chemistry:\n\n1. Principle of NMR spectroscopy: NMR spectroscopy is based on the principle that certain atomic nuclei, such as 1H (protons) and 13C (carbon-13), have a property called spin. When placed in a strong magnetic field, these nuclei can absorb and re-emit electromagnetic radiation at specific frequencies, which are dependent on the chemical environment of the nuclei. By analyzing the resulting NMR spectrum, chemists can obtain valuable information about the structure and composition

We can see the System and Instruction prompts are successfully masked!

In [17]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.203 GB of memory reserved.


In [18]:
trainer_stats = trainer.train(resume_from_checkpoint = resume_path)



The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,500 | Num Epochs = 1 | Total steps = 563
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


wandb: Initializing weave.
weave: Logged in as Weights & Biases user: georgz.
weave: View Weave data at https://wandb.ai/georgz-kth-royal-institute-of-technology/huggingface/weave


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
400,0.880300,0.931813


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▂▂▃▄▄▅▆▆▆▇██
train/global_step,▁▂▂▃▄▄▅▆▆▆▇██
train/grad_norm,▅▅▄▂▅▆█▂▄▁▁
train/learning_rate,██▇▆▅▄▃▂▂▁▁
train/loss,▆▅██▄▂▂▁▄▃▁
eval/loss,0.93181
eval/runtime,66.7202


In [19]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1336.3098 seconds used for training.
22.27 minutes used for training.
Peak reserved memory = 5.244 GB.
Peak reserved memory for training = 4.041 GB.
Peak reserved memory % of max memory = 35.574 %.
Peak reserved memory for training % of max memory = 27.413 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [20]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTo continue the Fibonacci sequence, we use the same rule that each number is the sum of the two preceding ones. \n\nStarting with 1, 1, 2, 3, 5, 8, the sequence would be:\n\n8, 13, 21, 34, 55, ']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [21]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

2, 4, 6, 10, 13, 17, 22,<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [22]:
model = model.merge_and_unload() # merge loraadapters

model.save_pretrained("lora_model_1b") # Local saving
tokenizer.save_pretrained("lora_model_1b")

######## SAVE locally before uploading #############

hugg_api = os.getenv('HUGG')

model.push_to_hub("gzsol/lora_model_1b", token = hugg_api) # Online saving
tokenizer.push_to_hub("gzsol/lora_model_1b", token = hugg_api) # Online saving



/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


README.md:   0%|          | 0.00/610 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...odel_1b/model.safetensors:   0%|          |  526kB / 1.10GB            

Saved model to https://huggingface.co/gzsol/lora_model_1b


README.md:   0%|          | 0.00/616 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  lora_model_1b/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [26]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = resume_path, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
A tall tower in the capital of France, likely Paris, stands out with its sleek and modern architecture. Constructed with a single column and four thinner columns on either side, the tower stands tall, giving an architectural impression with its sharp verticals and curved horizontal lines. The structure appears sturdy and resistant to damage, suggesting it can withstand heavy weather and possible earthquakes. This tall tower in Paris, like many in the city, symbolizes the city’s engineering prowess, and the unique blend of modern architec

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model_1b", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [27]:
hugg_api = os.getenv('HUGG')


# Merge to 16bit
model.save_pretrained_merged("model_1b", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("gzsol/model_1b", tokenizer, save_method = "merged_16bit", token = hugg_api)

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = hugg_api)

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("gzsol/model", tokenizer, save_method = "lora", token = hugg_api)

config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [01:01<00:00, 61.40s/it]


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...odel_1b/model.safetensors:   2%|1         | 41.9MB / 2.47GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:53<00:00, 113.56s/it]


Unsloth: Merge process complete. Saved to `/content/gzsol/model_1b`


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
from datetime import datetime

hugg_api = os.getenv('HUGG')

model_name = f"model_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Save to 8bit Q8_0
if False: model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "q8_0")
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if True: model.push_to_hub_gguf("hf/model", tokenizer, token = hugg_api)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf(model_name, tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        f"gzsol/{model_name}", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = hugg_api, # Get a token at https://huggingface.co/settings/tokens
    )


Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [01:19<00:00, 79.52s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:20<00:00, 80.44s/it]


Unsloth: Merge process complete. Saved to `/tmp/unsloth_gguf_qiag08qm`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages


Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>